# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

In [71]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder

data_path = '../data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [72]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


In [78]:
# 均值編碼 + 梯度提升樹
data = pd.concat([df[:train_num], train_Y], axis=1)
for c in df.columns:
    mean_df = data.groupby([c])['Survived'].mean().reset_index()
    mean_df.columns = [c, f'{c}_mean']
    data = pd.merge(data, mean_df, on=c, how='left')
    data = data.drop([c] , axis=1)
data = data.drop(['Survived'] , axis=1)
estimator = GradientBoostingRegressor()
#estimator = LinearRegression()
start = time.time()
print(f'shape : {data.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 1.0
time : 0.027573585510253906 sec


# 作業2
* 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?

ANS: 均值編碼。可能原因是Name和Cabin, Ticket等欄位使用標籤編碼後產生不具意函的數值大小關係，造成overfitting

In [79]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')
train_X.head()

shape : (891, 5)
score : 0.780004837244799
time : 0.03158688545227051 sec


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [82]:
# 均值編碼 + 邏輯斯迴歸
data = pd.concat([df[:train_num], train_Y], axis=1)
for c in df.columns:
    mean_df = data.groupby([c])['Survived'].mean().reset_index()
    mean_df.columns = [c, f'{c}_mean']
    data = pd.merge(data, mean_df, on=c, how='left')
    data = data.drop([c] , axis=1)
data = data.drop(['Survived'] , axis=1)
estimator = LogisticRegression()
#estimator = LinearRegression()
start = time.time()
print(f'shape : {data.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 1.0
time : 0.018078088760375977 sec


# My practice

In [73]:
def run_label_encoder (df):
    global train_num
    df_temp = pd.DataFrame()
    for c in df.columns:
        df_temp[c] = LabelEncoder().fit_transform(df[c])
    train_X = df_temp[:train_num]
    return train_X

def run_onehot_encoder (df):
    global train_num
    df_temp = df.drop('Name',axis=1)
    df_temp = df_temp.drop('Ticket',axis=1)
    df_temp = pd.get_dummies(df_temp)
    train_X = df_temp[:train_num]
    return train_X

def run_mean_encoder (df,train_Y):
    global train_num
    data = pd.concat([df[:train_num],train_Y],axis=1)
    #print(data.columns)
    #print(data)
    for c in df.columns:
#        print("data.groupby["+str(c)+"] = "+str(data.groupby([c]).mean()))
        mean_df = data.groupby([c])['Survived'].mean().reset_index()
        mean_df.columns = [c,f'{c}_mean']
        data = pd.merge(data,mean_df,on=c,how='left')
        data = data.drop([c],axis=1)
    return data.drop(['Survived'],axis=1)

In [75]:
result = {}
encoding=['label','onehot','mean']
regressor=['linear','GBR','Logistic']

for enc in encoding:
    if enc == 'label':
        XX = run_label_encoder(df)
    elif enc == 'onehot':
        XX = run_onehot_encoder(df)
    elif enc == 'mean':
        XX = run_mean_encoder(df,train_Y)
    result_enc = []
    for regr in regressor:
        if regr == 'linear':
            estimator = LinearRegression()
        elif regr == 'GBR':
            estimator = GradientBoostingRegressor()
        elif regr == 'Logistic':
            estimator = LogisticRegression()
        result_enc.append(cross_val_score(estimator, XX, train_Y, cv=5).mean())
    result[enc] = result_enc
result_df = pd.DataFrame(result)
result_df

,label,mean,onehot
0,0.334943,1.0,-3.183289e+25
1,0.377516,1.0,3.459370e-01
2,0.780005,1.0,7.833757e-01
